In [1]:
import mne
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.feature_selection import SelectKBest,f_classif,RFE  
from sklearn.svm import SVC 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt 
import seaborn as sns 
from FSRLearning import fsrlearning
from pathlib import Path 
from tqdm import tqdm 



# Load and preprocess EEG data 

In [2]:
def load_eeg_data(file_paths,freq_bands):
    data = []
    for path in tqdm(file_paths,desc="Loading EEG data"):
        raw = mne.io.read_raw_eeglab(path,preload=True)
        raw.filter(0.5,40)   # Bandpass filter
        psd, freqs = mne.time_frequency.psd_welch(raw, fmin=0.5, fmax=40)
        band_powers = []
        for band in freq_bands:
            idx = (freqs >= bnad[0] & freqs <= band[1])
            band_power = psd[:, idx].mean(axis=1)
            band_powers.append(band_power)
        data.append(np.concatenate(band_powers))
    return np.array(data)
        

In [3]:
# Load participants labels
def load_labels(participants_file):
    df = read_csv(participants_file,sep='\t')
    labels = df['group'].map({'PD':1,'Control':0}).values
    return labels

# Define frequency bands
freq_bands = [
    (1,4),  # Delta
    (4,8),  # Theta
    (8,12), # Alpha
    (12,30),# Beta
    (30,50), # Low Gamma
    (50,100) # High Gamma
    ]

In [4]:
# Load data
file_paths = list(Path('ds004584-download').glob('**/*.set'))
x = load_eeg_data(file_paths,freq_bands)
y = load_labels('ds004584-download/participants.tsv')

# Split data
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42, stratify=y)

# Standardize data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Feature selection using fsrlearning
fsrl = fsrlearning()
selected_features = fsrl.fit(x_train,y_train)
x_train_fsrl = x_train[:, selected_features]
x_test_fsrl = x_test[:, selected_features]




Loading EEG data:   0%|          | 0/149 [00:00<?, ?it/s]

Reading d:\new project\PD_&_EEG_Analysis\Parkinson_disease-EEG_Data_Analysis\ds004584-download\sub-001\eeg\sub-001_task-Rest_eeg.fdt
Reading 0 ... 140829  =      0.000 ...   281.658 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 s)



C:\Users\mdmnb\AppData\Local\Temp\ipykernel_35624\610458830.py:4: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(path,preload=True)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.1s finished
Loading EEG data:   0%|          | 0/149 [00:00<?, ?it/s]


AttributeError: No mne.time_frequency attribute psd_welch